In [15]:
import pymol
from pymol import cmd, stored
import numpy as np

# Start PyMOL
pymol.finish_launching()

In [2]:
def interfaceResidues(cmpx, cA='c. A', cB='c. B', cutoff=1.0, selName="interface"):
	"""
	interfaceResidues -- finds 'interface' residues between two chains in a complex.
	
	PARAMS
		cmpx
			The complex containing cA and cB
		
		cA
			The first chain in which we search for residues at an interface
			with cB
		
		cB
			The second chain in which we search for residues at an interface
			with cA
		
		cutoff
			The difference in area OVER which residues are considered
			interface residues.  Residues whose dASA from the complex to
			a single chain is greater than this cutoff are kept.  Zero
			keeps all residues.
			
		selName
			The name of the selection to return.
			
	RETURNS
		* A selection of interface residues is created and named
			depending on what you passed into selName
		* An array of values is returned where each value is:
			( modelName, residueNumber, dASA )
			
	NOTES
		If you have two chains that are not from the same PDB that you want
		to complex together, use the create command like:
			create myComplex, pdb1WithChainA or pdb2withChainX
		then pass myComplex to this script like:
			interfaceResidues myComlpex, c. A, c. X
			
		This script calculates the area of the complex as a whole.  Then,
		it separates the two chains that you pass in through the arguments
		cA and cB, alone.  Once it has this, it calculates the difference
		and any residues ABOVE the cutoff are called interface residues.
			
	AUTHOR:
		Jason Vertrees, 2009.		
	"""
	# Save user's settings, before setting dot_solvent
	oldDS = cmd.get("dot_solvent")
	cmd.set("dot_solvent", 1)
	
	# set some string names for temporary objects/selections
	tempC, selName1 = "tempComplex", selName+"1"
	chA, chB = "chA", "chB"
	
	# operate on a new object & turn off the original
	cmd.create(tempC, cmpx)
	cmd.disable(cmpx)
	
	# remove cruft and inrrelevant chains
	cmd.remove(tempC + " and not (polymer and (%s or %s))" % (cA, cB))
	
	# get the area of the complete complex
	cmd.get_area(tempC, load_b=1)
	# copy the areas from the loaded b to the q, field.
	cmd.alter(tempC, 'q=b')
	
	# extract the two chains and calc. the new area
	# note: the q fields are copied to the new objects
	# chA and chB
	cmd.extract(chA, tempC + " and (" + cA + ")")
	cmd.extract(chB, tempC + " and (" + cB + ")")
	cmd.get_area(chA, load_b=1)
	cmd.get_area(chB, load_b=1)
	
	# update the chain-only objects w/the difference
	cmd.alter( "%s or %s" % (chA,chB), "b=b-q" )
	
	# The calculations are done.  Now, all we need to
	# do is to determine which residues are over the cutoff
	# and save them.
	stored.r, rVal, seen = [], [], []
	cmd.iterate('%s or %s' % (chA, chB), 'stored.r.append((model,resi,b))')

	cmd.enable(cmpx)
	cmd.select(selName1, 'none')
	for (model,resi,diff) in stored.r:
		key=resi+"-"+model
		if abs(diff)>=float(cutoff):
			if key in seen: continue
			else: seen.append(key)
			rVal.append( (model,resi,diff) )
			# expand the selection here; I chose to iterate over stored.r instead of
			# creating one large selection b/c if there are too many residues PyMOL
			# might crash on a very large selection.  This is pretty much guaranteed
			# not to kill PyMOL; but, it might take a little longer to run.
			cmd.select( selName1, selName1 + " or (%s and i. %s)" % (model,resi))

	# this is how you transfer a selection to another object.
	cmd.select(selName, cmpx + " in " + selName1)
	# clean up after ourselves
	cmd.delete(selName1)
	cmd.delete(chA)
	cmd.delete(chB)
	cmd.delete(tempC)
	# show the selection
	cmd.enable(selName)
	
	# reset users settings
	cmd.set("dot_solvent", oldDS)
	
	return rVal

cmd.extend("interfaceResidues", interfaceResidues)

<function __main__.interfaceResidues(cmpx, cA='c. A', cB='c. B', cutoff=1.0, selName='interface')>

In [3]:
cmd.load('../data/scan_complex_pdbs/P17028_P28698_scand_complex.pdb')

1

In [4]:
interactor_residues = interfaceResidues("P17028_P28698_scand_complex", "chain A", "chain B", 1.0, "interface_residues")

 PyMOL(TM) 3.0.1 - Incentive Product
 Copyright (C) Schrodinger, LLC
 
 This Executable Build integrates and extends Open-Source PyMOL.
No License File - For Evaluation Only (29 days remaining)
 Detected OpenGL version 4.1. Shaders available.
 ShaderPrg-Error: vertex shader compilation failed.; name='bezier'
 ShaderPrg-Error-InfoLog:
0:1(10): error: GLSL 4.50 is not supported. Supported versions are: 1.10, 1.20, 1.30, 1.40, 1.50, 3.30, 4.00, 4.10, 1.00 ES, and 3.00 ES

 Detected GLSL version 4.10.
 OpenGL graphics engine:
  GL_VENDOR:   Microsoft Corporation
  GL_RENDERER: D3D12 (Intel(R) UHD Graphics)
  GL_VERSION:  4.1 (Compatibility Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
 ShaderPrg-Error: vertex shader compilation failed.; name='bezier'
 ShaderPrg-Error-InfoLog:
0:1(10): error: GLSL 4.50 is not supported. Supported versions are: 1.10, 1.20, 1.30, 1.40, 1.50, 3.30, 4.00, 4.10, 1.00 ES, and 3.00 ES


 Detected 12 CPU cores.  Enabled multithreaded rendering.


In [5]:
cmd.color("red", "interface_residues")

1

In [9]:
interactor_residues

[('chA', '4', 4.488032341003418),
 ('chA', '5', 11.351110458374023),
 ('chA', '7', 15.991887092590332),
 ('chA', '8', 3.732332706451416),
 ('chA', '9', 2.128385066986084),
 ('chA', '15', 4.068734645843506),
 ('chA', '16', 1.3074989318847656),
 ('chA', '18', 2.63851261138916),
 ('chA', '19', 5.345416069030762),
 ('chA', '22', 6.577596187591553),
 ('chA', '23', 3.2766122817993164),
 ('chA', '26', 8.623576164245605),
 ('chA', '27', 5.3726091384887695),
 ('chA', '29', 1.6003272533416748),
 ('chA', '30', 4.625360488891602),
 ('chA', '32', 15.909313201904297),
 ('chA', '33', 5.228797912597656),
 ('chA', '35', 11.252552032470703),
 ('chA', '36', 6.612647533416748),
 ('chA', '37', 2.275022506713867),
 ('chA', '38', 13.083211898803711),
 ('chA', '40', 7.390873908996582),
 ('chA', '41', 4.510503768920898),
 ('chA', '42', 19.555404663085938),
 ('chA', '44', 6.502278804779053),
 ('chA', '45', 3.6973764896392822),
 ('chA', '47', 7.314125061035156),
 ('chA', '48', 2.2392420768737793),
 ('chA', '49',

In [17]:
selected_numbers = [t[1] for t in interactor_residues if t[0] == 'chA']
np.array(selected_numbers, dtype=int) + np.array([44]*len(selected_numbers))

array([ 48,  49,  51,  52,  53,  59,  60,  62,  63,  66,  67,  70,  71,
        73,  74,  76,  77,  79,  80,  81,  82,  84,  85,  86,  88,  89,
        91,  92,  93,  96,  97,  98, 101, 113, 114, 117, 118, 120, 121,
       124])